In [1]:
from gurobipy import*
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt
import pandas
from itertools import combinations
import math
import csv
import pandas as pd
from rsome import dro

In [2]:
def solvePerfectKnowledge(m,n, Dt ,Kch, Kdis, UB_x, UB_y,PriceImp, PriceEx, SoEmin, SoEmax, SoEprev,Pprod, Pcons, obj_fct='min_cost'):
  

  #create a model object
  model = dro.Model('model')

  #decision variables
  Pimp = model.dvar(m)          #Imported Power                
  Pexp = model.dvar(m)          #Exported Power
  Bch = model.dvar((m,n))       #Charge the battery         
  Bdis = model.dvar((m,n))      #Discharge the battery
  SoE = model.dvar((m,n))       #State of Energy (of the battery)
  #b = model.dvar(m, 'B')
  Zimp  = model.dvar(m, 'B')
  Zexp  = model.dvar(m, 'B')
  #c = model.dvar((m,n), 'B')
  Zch = model.dvar((m,n), 'B')
  Zdis = model.dvar((m,n), 'B')
  SoEnext = model.dvar((m,n))   #State of Energy of the battery in the next time timeslot


  #define the two objective functions/startegies
  if obj_fct == 'min_cost': #cost minimization strategy/profit maximization 
    model.min((PriceImp*Pimp).sum() - (PriceEx * Pexp).sum())


  #define the Constraints
  
  model.st(((sum(Pprod[t]) - Bch.sum(axis = 1) + Bdis.sum(axis = 1) + Pimp[t] - Pexp[t]) >=  sum(Pcons[t])) for t in range(0,m))

  #2. No import and export at the same time
  model.st(Pimp[t] <= Zimp[t]* UB_x for t in range(0,m))   #constraint (2) x <= (1-b) * UB_x  ,y <= b * UB_y
  model.st(Pexp[t] <= Zexp[t] * UB_y for t in range(0,m) ) #Ub_x : maximum import, UB_y : maximum import
  model.st(Pimp >=0 ) 
  model.st(Pexp >=0 )
  model.st(Zimp[t] + Zexp[t] <= 1 for t in range(0,m))
  #3.ESS constraints, i.e, battery constraints
  model.st( SoE[0] == SoEprev ) #initialize the state of Energy of the battery
  model.st( SoEnext[t] == SoE[t] + (Bch[t] - Bdis[t]) * Dt for t in range(0,m))
  model.st( SoE[t] == SoEnext[t-1] for t in range(1,m-1))
  model.st( SoE[m-1] == SoEnext[m-2])
  model.st( SoEnext <= SoEmax) # SoE maximum capacity
  model.st( SoEnext[t] >= SoEmin[t] for t in range(0,m) ) #SoE minimum capacity
    
  model.st(Bch>=0)
  model.st(Bch[t] <= Kch[t] for t in range(0,m)) #Battery charging limits
  model.st(Bdis>=0)
  model.st(Bdis[t] <= Kdis[t] for t in range(0,m)) #Battery discharging limits
  model.st(Bch[t] <= Zch[t] * Kch[t] for t in range(0,m)) #maximum charging/discharging limit #(80% of maximum capacity for example)
  model.st(Bdis[t] <= Zdis[t] * Kdis[t] for t in range(0,m))
  model.st(Zch[t] + Zdis[t] <= 1 for t in range(0,m))


   
    
  ############################# SOLVING ############################
  print('SOLVING')  #Use the solver gurobi to solve the optimization problem , GENERATE THE LOG FILE for more details about the solving process
  #model.solve(grb) #solve the problem using Gurobi solver
  model.solve(solver=grb, display=True)
    #'DualReductions':0
  
  #model.solve(solver=grb, display=True, params={'LogFile': "/home/jovyan/work/SeptemberFinal/FABIO_DETERMINISTIC/deterministic_logfile_INEQ" , 'ResultFile': "/home/jovyan/work/SeptemberFinal/FABIO_DETERMINISTIC/solution_INEQ.sol",'OutputFlag': 1, "LogToConsole" : 1})
  formula = model.do_math()
  print(formula)
    
  obj_value = model.get()
  Pimport = Pimp.get()
  Pexport = Pexp.get()
  Bcharge = Bch.get()
  Bdischarge = Bdis.get()
  Soe = SoE.get()
  soenext = SoEnext.get()


  save_results(Pprod,Pcons,Soe,Bcharge,Bdischarge,obj_value,Pimport,Pexport,PriceImp,PriceEx, n = 2, m=24)




  return(model.get(),Pimp.get(),Pexp.get(),Bch.get(),Bdis.get(),SoE.get(),SoEnext.get()) #return the objective value and the decision variables optimal values


In [11]:
#extract the dataframe for the production and consumption input data (city learn) which is considered as the ground truth for our simulation
#using city learn data #City Learn data is the actual data
def InputParameters(filename, n , m ): #9 buildings 24h
    #extract all the production and consumption values from the csv file



    #merged_f.to_csv(filename)
    merged_f = pd.read_csv('combined_energy_data.csv')

    #df_mh is the the dataframe for m timesteps
    df_mh = merged_f.iloc[4379:4379+m,0:] #m=24
    d_consumption = df_mh[['Blow_Consumed_kWh','Schutz_Consumed_kWh']] #4:4+n
    d_production = df_mh[['Blow_Produced_kWh','Schutz_Produced_kWh']] #4+n: (4+2*n)
    c1 = d_consumption.to_numpy()
    p1 = d_production.to_numpy()
            
    return p1,c1 #,s1 #rerurn production and consumption of city learn

In [12]:
#extract the dataframe containing the import and the export prices for m timesteps
#Customized Prices from the Nordpool
def Prices(filename):

    prices = pd.read_csv('df_year_price.csv')
    Price_imp_df = prices['import'] #24 hours of prices from nord pool 
    #converting the dataframe to a numpy array
    PriceImp=Price_imp_df[4379:4403]
    PriceImp = PriceImp.to_numpy()
    #l = Price_imp
    #Price_exp = Price_imp
    #Price_exp[0:5] = Price_imp[0:5] *2 #we want to import at night, no peak, less stress, charge vehicle or battery, 
    #Price_exp[5:10] = Price_imp[5:10]/3 #prices of import start to increase due demand
    #Price_exp[10:20] = Price_imp[10:20]/5 #price of import keep increasing due demand increase
    #Price_exp[20:24] = Price_imp[20:24]*3 #
    Price_exp_df = prices['export'] #24 hours of prices from nord pool 
    PriceEx=Price_exp_df[4379:4403]
    PriceEx = PriceEx.to_numpy()

    
    
    return PriceEx,PriceImp

In [13]:
prices = pd.read_csv('df_year_price.csv')
Price_imp_df = prices['import'] #24 hours of prices from nord pool 
    #converting the dataframe to a numpy array
PriceImp=Price_imp_df[4379:4403]
PriceImp = PriceImp.to_numpy()
    #l = Price_imp
    #Price_exp = Price_imp
    #Price_exp[0:5] = Price_imp[0:5] *2 #we want to import at night, no peak, less stress, charge vehicle or battery, 
    #Price_exp[5:10] = Price_imp[5:10]/3 #prices of import start to increase due demand
    #Price_exp[10:20] = Price_imp[10:20]/5 #price of import keep increasing due demand increase
    #Price_exp[20:24] = Price_imp[20:24]*3 #
Price_exp_df = prices['export'] #24 hours of prices from nord pool 
PriceEx=Price_exp_df[4379:4403]
PriceEx = PriceEx.to_numpy()

In [14]:
 PriceEx

array([0.04006, 0.01854, 0.03681, 0.04927, 0.05297, 0.09865, 0.12727,
       0.16807, 0.16456, 0.13454, 0.11755, 0.0608 , 0.0066 , 0.00727,
       0.00753, 0.00802, 0.00965, 0.02456, 0.04002, 0.18283, 0.14461,
       0.10683, 0.06087, 0.05074])

In [15]:
def main():
    m = 24
    n = 2
    #battery_capacity = np.array([140,80,50,75,50,30,40,30,35]) #maximum amount of electrical energy that the battery can store (kWh)
    #randnums = np.array([40,64,10,45,32,20,16,10,30]) #initial SoE (kWh)
    battery_capacity_max = [8700,8700] #battery maximum capacity
    battery_capacity = np.array(battery_capacity_max) #maximum amount of electrical energy that the battery can store (kWh)
    randnums = np.array([10,25]) #initial SoE (kWh)


    Dt = 1 #24*1 #24h   A verifier avec A.T ou bien 1h


   
    SoEmin_ = battery_capacity*0.2 #50% of the battery_capacity
    SoEmax_ = battery_capacity*0.90 #98% of the battery_capacity


    SoEmin = np.array( [ SoEmin_ for i in range(0,m)])  #minimum of SoE (Energy kWh) 10% of battery_capacity
    SoEmax= np.array([ SoEmax_  for i in range(0,m)]) #maximum of SoE  (Energy kWh) 90% of battery_capacity
    
    SoEprev = np.array([8700*0.5,8700*0.5])

    #SoEprev = randnums #(Energy Kwh) initialize SoE to previous timestep values

   

    Kch = SoEmax #maximum charging
    Kdis = SoEmax #maximum discharging 

    #upper and lower bounds for the import and export

    UB_x = 10000 #maximum import ###
    UB_y = 10000 #maximum export  ####


    
 
    #Get the input Production and Consumption
   
    Pprod, Pcons = InputParameters(filename= "combined_energy_data.csv", n = 2, m = 24) #the input csv file contains data about 250 prosumers for 12 tiesteps. we use n1 prosumers for m timesteps
    PriceEx,PriceImp = Prices(filename = "df_year_price.csv")

     
    obj_value,Pimport,Pexport,Bcharge,Bdischarge,Soe,soenext = solvePerfectKnowledge(m,n, Dt ,Kch, Kdis,
                          UB_x, UB_y,PriceImp, PriceEx, SoEmin, SoEmax, SoEprev,Pprod,
                          Pcons, obj_fct='min_cost')
    
    return (obj_value,
                 Pimport,
                 Pexport,
                  Bcharge,
                 Bdischarge,
                 Soe,
                 soenext)
   

In [16]:
#we use the predicted data
def save_results(Pprod,Pcons,Soe,Bcharge,Bdischarge,obj_value,Pimport,Pexport,PriceImp,PriceEx, n = 2, m=24):

  #save prosumers results
    #l= 0
    #for gamma in range(0,n):
        p= 3

        #obj_value,Pimport,Pexport,Bcharge,Bdischarge,Soe,soenext = solvePartialKnowledge(gamma, dev,m,n, capacity, Kch, Kdis, UB_x,UB_y,PriceImp, PriceEx,SoEmin, SoEmax, Bch_prev, Bdis_prev, SoEprev,production, consumption, obj_fct='min_cost')
        k=0 
        Timestep = [None] *n*m
        Prosumer = [None] *n*m
        d = [0.1] *n*m
        for i in range(0,m):
            k+= 1
            p=1
            for j in range(i*n, (i+1)*n):
                Timestep[j] = k
                Prosumer[j] = p
                p+= 1

        Production = (np.reshape(Pprod, (1,n*m)))[0] #reshape? #nominal
        Consumption = (np.reshape(Pcons , (1,n*m)))[0]  #nominal
        SurplusDeficit = Pprod - Pcons
        SurplusOrDeficit = (np.reshape(SurplusDeficit,(1,n*m) ))[0]
        StateOfEnergy = (np.reshape(Soe, (1,n*m)))[0]
        Charge_Battery = (np.reshape(Bcharge, (1,n*m)))[0]
        Discharge_Battery = (np.reshape(Bdischarge, (1,n*m)))[0]

        #Charge_Battery = Bch.get()
        #Discharge_Battery = Bdis.get()
        df = pandas.DataFrame(data={"Timestep": Timestep, "Prosumer": Prosumer,  "Production_Nominal": Production, "Consumption_Nominal": Consumption, "SurplusOrDeficit_Nominal": SurplusOrDeficit,
                                    "StateOfEnergy": StateOfEnergy, "Charge_Battery": Charge_Battery, "Discharge_Battery": Discharge_Battery})
        df.to_csv( "Model_1_Prosumer_SIVA_for_24hours_.csv", sep=',',index=False)

        #save MG results
        
        #--------------------------
        ### Microgrid data ###
        Timesteps = [None] *m
        MGprod = [None] *m
        MGpcons = [None] *m
        MG_SurplusOrDeficit = [None] *m
        MGexport = [None] *m
        MGimport = [None] *m
        MGcharge = [None] *m
        MGdischarge = [None] *m
        MGsoe = [None] *m
        MGcost_profit = [None] *m
        s = 0
        for i in range(0,m):
            s+=1
            Timesteps[i] = s
            MGprod[i] = Pprod[i].sum() #Total MG production for every timestep
            MGpcons[i] = Pcons[i].sum() #Total MG consumption for every timestep
            MG_SurplusOrDeficit[i] = MGprod[i] - MGpcons[i] #MG surplus or deficit for every time step
            MGimport[i] = Pimport[i] #Total MG import for every timestep
            MGexport[i] = Pexport[i] #Total MG export for every timestep
            MGcharge[i] = Bcharge[i].sum() #Total MG Battery charge for every timestep
            MGdischarge[i] = Bdischarge[i].sum() #Total MG Battery discharge for every timestep
            MGsoe[i] = Soe[i].sum() #Total MG Battery SOE for every timestep
            MGcost_profit[i] = PriceImp[i]*MGimport[i] - PriceEx[i] *MGexport[i]
            #Add total cost/profit

        df = pandas.DataFrame(data={"Timestep": Timesteps,  "MG_nominal_Production": MGprod, "MG_nominal_Consumption": MGpcons, "MG_SurplusOrDeficit": MG_SurplusOrDeficit,
                                    "MG_Export": MGexport, "MG_import": MGimport, 
                                     "MG_Charge_Battery": MGcharge, "MG_Discharge_Battery": MGdischarge, "MG_StateOfEnergy": MGsoe,
                                     "MGcost_profit": MGcost_profit})
        df.to_csv("Model_1_MG_SIVA_for24hours.csv", sep=',',index=False)
        #l+= 1






In [17]:
n=2
m=24
if __name__ == "__main__":
    main()

SOLVING
Set parameter Username
Academic license - for non-commercial use only - expires 2025-07-09
Being solved by Gurobi...
Solution status: 2
Running time: 0.0990s
Conic program object:
Number of variables:           386
Continuous/binaries/integers:  242/144/0
---------------------------------------------
Number of linear constraints:  6126
Inequalities/equalities:       5646/480
Number of coefficients:        22797
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0
---------------------------------------------
Number of PSCone constraints:  0

